## Project 4-2 : Data Acquisition SQL

## Assignment Instructions

### Description

Some SQL problems.

### Grading

For grading purposes, we will clear all outputs from all your cells and then run them all from the top.  Please test your notebook in the same fashion before turning it in.

### Submitting Your Solution

To submit your notebook, first clear all the cells (this won't matter too much this time, but for larger data sets in the future, it will make the file smaller).  Then use the File->Download As->Notebook to obtain the notebook file.  Finally, submit the notebook file on Canvas.

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from sqlalchemy import create_engine, inspect

### Problem: SQL (50 points)

Use the csci303 database downloaded in class and code provided (12-SQL_New) to answer the following questions.  Note that you can answer every one of these questions just with single SQL queries, but you are free to instead pull data into pandas and answer the questions that way, or use multiple queries, etc.

The data you'll be working with is from the [Bureau of Labor Statistics](https://www.bls.gov/emp/documentation/education-training-system.htm).
<br>There are three tables:
- `occupation_codes` provides a mapping between two different organization's occupation codes and titles.  You'll need this table to lookup occupation codes for the other two tables.
- `occupation_attainments` gives the percentages of workers (aged 25 or older) in each occupation (in 2014-2015) who had attained a particular level of education.
- `occupation_assignments` gives requirements for each occupation in terms of education, experience, and on-the-job training.

The code to inspect the names and types of the columns in thses tables is given below. Feel free to comment this out once you get a feel for the table structures.

## SET UP

To set up we have to create an inspector on the database to get the information we need! To do so we will follow these steps:
1. Create a Database variable, this holds the name of the database you downloaded for this assignment!
2. Create an inspector object and create_engine on the database object from # 1
3. Print out information from the database to gain a better understanding of what it contains!

---

**Questions for Set Up (Edit this cell):**

---

(a) What is the variable type of the acs_code column? <font color = blue>Integer</font>

(b) What does the 'Real' column type mean? <font color = blue>A real number with decimal places</font>

(c) What column(s) are shared amongst the three tables? This is how they are related!! <font color = blue> soc_code</font>

In [ ]:
#FINISH THIS CODE

dburi = 'sqlite:///csci303.sqlite3' # Data base connection!
inspector = inspect(create_engine(dburi)) # Here we have created an inspector to take a look at the db!

#lamda function! 
tables = list(filter(lambda name: "occupation" in name, inspector.get_table_names()))

#returns list of all tables in a database
# inspector.get_table_names('main')

#returns list of tables with "occupation" in the name from database
tables

In [ ]:
# pd.read_sql_query("SELECT * FROM occupation_assignments", create_engine(dburi))

In [ ]:
# pd.DataFrame(inspector.get_columns('occupation_codes','main'))
# pd.DataFrame(inspector.get_columns('occupation_codes'),columns=['name','type'])

In [ ]:
# Loop through the tables list and print all of the table names and columns
for table in tables:
    print("Table name: ", table) # Print the table name (table)
    for column in inspector.get_columns(table): # Print the column name and type, think of the column as a dataframe, column['xxx']
        print("Column name: %s: %s" % (column['name'], column['type'])) # TODO enter in what you want from the columns
    print()
    
#we use 'name' and 'type' because looking at the questions above they ask about what data TYPE is a specific column (NAME).
#they also ask what are the common column names (we could use code to count probably, but eyes are easier here)

#soc_code 3, thats it

---

**Questions for Problem 2 (Edit this cell):**

---

(a) What are the SOC and ACS codes for 'Database administrators ' - (this may have a space at the end)?

<font color = blue>soc_code 15-1141	       acs_code	1060</font>


(b) How many SOC occupation titles are there grouped under the ACS title 'Postsecondary teachers'?

<font color = blue>38</font>

(c) How much education and prior experience do you typically need to become a software developer?

<font color = blue>First I had to go understand how the data was being presented for occupation_assignments... I understand now each value reported under the education level is a % of people in that field with that degree level. 
    https://www.bls.gov/emp/tables/educational-attainment.htm

So, it looks like roughly 49% have Bachelors and 30% have Masters. However, it appears that only a bachelor's with no previous experience is *required* in order to get a job.</font>

(d) What SOC occupation has the highest percentage of doctoral or professional degree holders? 

<font color = blue>Podiatrists</font>

EXTRA CREDIT: (5pts) What acs_title has the longest length? (There are two right answers!)

<font color = blue>"Miscellaneous food preparation and serving related workers, including dining room and cafeteria attendants and bartender helpers" if we go by character count (with or without spaces)

"Broadcast and sound engineering technicians and radio operators, and media and communication equipment workers, all other" could tie the former in word count (both 16), but not in characters.
    
Is this what was referred to when you said two right answers? Or just the fact that the entries get duplicated?

</font>



In [ ]:
#(a) What are the SOC and ACS codes for 'Database administrators ' - (this may have a space at the end)?

query = """
SELECT soc_title, soc_code, acs_title, acs_code
FROM  occupation_codes
WHERE soc_title LIKE 'Database administrators%%' or acs_title LIKE 'Database administrators%%'
"""

pd.read_sql_query(query, dburi)

In [ ]:
#(b) How many SOC occupation titles are there grouped under the ACS title 'Postsecondary teachers'?

#this will show you all the individual soc titles for the acs title
# query = """
# SELECT soc_title, soc_code, acs_title, acs_code
# FROM  occupation_codes
# WHERE acs_title IS 'Postsecondary teachers'
# ORDER by soc_title
# """

# pd.read_sql_query(query, dburi)


query = """
SELECT acs_title, COUNT(DISTINCT soc_title) AS title_cnt
FROM  occupation_codes
WHERE acs_title IS 'Postsecondary teachers'
ORDER by soc_title
"""

pd.read_sql_query(query, dburi)

In [ ]:
#(c) How much education and prior experience do you typically need to become a software developer?

#first find the soc_title
query = """
SELECT soc_title, acs_title, soc_code
FROM  occupation_codes
WHERE acs_title LIKE '%%software%%' OR soc_title LIKE '%%software%%'
"""

pd.read_sql_query(query, dburi)

In [ ]:
#Let's see what the job requirements are...
query = """
SELECT *
FROM  occupation_assignments
WHERE soc_code IS '15-1132' OR soc_code IS '15-1133'
"""

pd.read_sql_query(query, dburi)

In [ ]:
#so lets look at the actual count of degrees in that job
query = """
SELECT *
FROM  occupation_attainments
WHERE soc_code IS '15-1132' OR soc_code IS '15-1133'
"""

pd.read_sql_query(query, dburi)

In [ ]:
#(d) What SOC occupation has the highest percentage of doctoral or professional degree holders?

query = """
SELECT ass.soc_code, soc_title, doctoral_or_professional
FROM  occupation_attainments AS att JOIN occupation_codes AS cod 
ON att.soc_code = cod.soc_code
JOIN occupation_assignments AS ass
ON cod.soc_code = ass.soc_code
ORDER by doctoral_or_professional DESC
"""

pd.read_sql_query(query, dburi)

In [ ]:
#EXTRA CREDIT: (5pts) What acs_title has the longest length? (There are two right answers!)

query = """
SELECT DISTINCT acs_title, LENGTH(acs_title) AS length
FROM  occupation_codes
ORDER by length DESC
"""

pd.set_option('display.max_colwidth', None)
pd.read_sql_query(query, dburi)

#First) 128 characters, 113 no spaces, 16 words
#Second) 122 c, 109 ns, 14 words
#Third) 121 c, 106 ns, 16 words

### Questionnaire
1) How long did you spend on this assignment? 

<font color = blue> An hour to two.</font>
<br><br>
2) What did you like about it? What did you not like about it?
    
<font color = blue>I liked the progression of using SQL through the problems. Looking back I could probably solve C in one SQL query, however I felt like I got the practice from doing it in D. The syntax of SQL is strange, but I find it more intuitive and fun to work with than using typical python language for finding information. </font>
<br><br>
3) Did you find any errors or is there anything you would like changed?
  
<font color = blue> Honestly, I could have done one or two more problems. By the time I got to D I felt like I was on a roll and the problem was much easier to handle than C. </font>
<br><br>

In [ ]:
#I decided I would attemtp to do C in one code...
#(c) How much education and prior experience do you typically need to become a software developer?


query = """
SELECT cod.acs_title, ass.education, ass.experience, att.*
FROM  occupation_attainments AS att JOIN occupation_codes AS cod 
ON att.soc_code = cod.soc_code
JOIN occupation_assignments AS ass
ON cod.soc_code = ass.soc_code
WHERE cod.acs_title LIKE '%%software%%'
"""

pd.read_sql_query(query, dburi)